<div style="text-align: center"><h1><font size='10' face = "Comic sans MS" color="#008e94">HappierWhale 🐳 </font></h1></div>
<div style="text-align: center"><h1><font size='6' face = "Comic sans MS" color="#008e94">3% Annotated!</font></h1></div>

<div class="alert alert-block alert-info"> 
    Please upvote if you find this helpful. It will help me to indentify what is helpful in a competition and what it not. Thanks!
</div>

Hello! This notebook is to give you an overview of the "HappierWhale" dataset, containing BB annotations for ~1500 images. 🕺🏻   
      
The scope of the notebook is to walk you through all the steps, so you can decide whether or not you want to use/explore it.  

Yes, I could have trained a model on this and shared all cropped train images. I shall give it a go, but meanwhile if someone can do it better with longer training, we can all benefit from a strong model!  
    
    
Hope this helps the challenge a little, Cheers!🍻

# 🌟 Motivation

By now most of us understood that this is a pretty challenging task with exteremely diverse training images from different distributions, views, time periods and so on. Comparitively, the dataset from the previous challenge of Re-ID with flukes was insanely clean. Even then top solutions had to use manual annotations to improve it, so maybe this does too.

There are couple cropped datasets, thanks to their efforts and for sharing them with others. Especially the one from @phalanx [link to dataset](https://www.kaggle.com/phalanx/whale2-cropped-dataset) using Facebook Detic seems to be a reliable approach. 



# 🏷 Labelling 

Labelling these ~1500 images, I realised that **many** of the samples lack any distinct features to id or even classify them among the 26 species. Again comparing to the flukes dataset, its highly inconsistent. Using **all** the training images for contrastive learning seems like a bad idea. Unlike the last competition, this seems to be about decreasing the training samples to have a more consistent and learnable data. 😅

I initally started with labelling fin, body, zoomed-out etc but quickly gave up and just fixed to single class. Perhaps you can use the species label as class when training detector but you mostly likely would end up confusing the mode. The code used to select these samples is in the following section.

I have to confess that the labelling was not consistent. Some contain just fin or fin plus part of the body and sometimes the whole body depending upon the pattern or what other images focused on. Some examples would give you a better idea.

In [ ]:
import pandas as pd
import numpy as np
import cv2
from skimage import io
from skimage.color import gray2rgb
from typing import Tuple
import os
import shutil
import plotly.express as px
import plotly.graph_objects as go
from skimage.transform import resize

seed = 400

subset_1_df = pd.read_csv("/kaggle/input/happierwhale/subset_1_df.csv")
subset_1_df.describe(include='all').loc[['count', 'unique', 'freq', 'top']]

In [ ]:
annotation_files = os.listdir("/kaggle/input/happierwhale/yolo_annotations/yolo_annotations/")
print("Number of annotations - ", len(annotation_files))
print("Number of samples - ", len(subset_1_df))

One species with too samples to get 3% from did not make it 😕  
18 samples not labelled!

# 🦈 Annotations

In [ ]:
read_img = lambda file: cv2.imread(f"/kaggle/input/happy-whale-and-dolphin/train_images/{file}")
read_ann = lambda file: np.loadtxt(f"/kaggle/input/happierwhale/yolo_annotations/yolo_annotations/{file}")

# easier to draw with cv, ref - https://stackoverflow.com/questions/64096953/how-to-convert-yolo-format-bounding-box-coordinates-into-opencv-format
def get_ann_image(ann_file):
    image_file = ann_file.split(".txt")[0] + ".jpg"
    img = read_img(image_file)
    dh, dw, _ = img.shape
    _, x, y, w, h = read_ann(ann_file)
    l = int((x - w / 2) * dw)
    r = int((x + w / 2) * dw)
    t = int((y - h / 2) * dh)
    b = int((y + h / 2) * dh)
    if l < 0:
        l = 0
    if r > dw - 1:
        r = dw - 1
    if t < 0:
        t = 0
    if b > dh - 1:
        b = dh - 1
    cv2.rectangle(img, (l, t), (r, b), (0, 255, 255), 8)
    img = cv2.resize(img, (300,300), interpolation = cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

In [ ]:
# sample and plot annotations
annotation_df = pd.DataFrame(annotation_files)
example_annotations = annotation_df.sample(10, random_state=seed).reset_index(drop=True)
annotated_images = np.asarray([get_ann_image(x) for x in example_annotations[0]])

fig = px.imshow(annotated_images, facet_col=0, facet_col_wrap=5, binary_string=True, facet_row_spacing=0.0, facet_col_spacing=0)
fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
fig.update_layout(title="Sample Annotations", autosize=True, hovermode=False, height=900, width=1500, margin=dict(l=0, r=0, t=150, b=80))
fig.show()

# Skipped Images 

In [ ]:
# intermediate output of modified script of https://gist.github.com/Amir22010/a99f18ca19112bc7db0872a36a03a1ec
missed_samples = ['8c5221ff535f10.jpg', 'cf4f3fe6605328.jpg', 'f8ec1df54bba10.jpg', '9c96a72250084c.jpg', '9a8b39b1b6d472.jpg', '60de2ea6b649a8.jpg', 'dda4bba985a0f0.jpg', 'c45d288419aba6.jpg', 'dd46ff0597c1aa.jpg', 'fdf98dfb01d768.jpg', 'baf82000789858.jpg', 'ebc7e212d6e505.jpg', 'a8e7f0347b7c62.jpg', 'a6d9a9d4a8a444.jpg', 'b58f4f1d9f6247.jpg', 'abe8de4c14ddbc.jpg', 'aa16947ce5316e.jpg', 'af64d05f30daca.jpg']

read_img = lambda file: io.imread(f"/kaggle/input/happy-whale-and-dolphin/train_images/{file}")
missed_imgs = []
for x in missed_samples:
    img = read_img(x)
    if len(img.shape) == 2: 
        img = gray2rgb(img)
    missed_imgs.append(resize(img, (300,300)))
missed_imgs = np.asarray(missed_imgs)

# plot image grid
fig = px.imshow(missed_imgs, facet_col=0, facet_col_wrap=10, binary_string=True, facet_row_spacing=0.0, facet_col_spacing=0)
fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
fig.update_layout(title="Missed/skipped images", autosize=True, hovermode=False, height=900, width=1500, margin=dict(l=0, r=0, t=150, b=80))
fig.show()

Bad views and last but one was a mistake, donno how that happened, its fine 😅 but this doesnt mean all that are annotated are better samples than these

# 👨🏻‍💻 Sampling Code 

```
# just read the train csv with known corrections
def get_train_csv(
    path:str="/kaggle/input/happy-whale-and-dolphin/train.csv"
    )->Tuple[pd.DataFrame, pd.DataFrame]:
    # fix spelling of two species and merging subspecies, reduces the unique species from 30 -> 26
    train_df = pd.read_csv(path)
    train_df['species'].replace({'bottlenose_dolpin': 'bottlenose_dolphin', 
                                 'kiler_whale': 'killer_whale',
                                 'pilot_whale': 'short_finned_pilot_whale',
                                 'globis': 'short_finned_pilot_whale'
                                }, inplace=True)

    # assign id to species
    train_df['species_id'], species_index = train_df['species'].factorize()
    return train_df, species_index

# df with all samples
train_df, species_index = get_train_csv(path="/kaggle/input/happy-whale-and-dolphin/train.csv")
train_df.describe(include='all').loc[['count', 'unique', 'freq', 'top']]

# sampling ~3% from every species randomly
subset_1_df = train_df.groupby('species').apply(pd.DataFrame.sample, frac=0.03, random_state=seed).reset_index(drop=True)
subset_1_df.describe(include='all').loc[['count', 'unique', 'freq', 'top']]

# save the df and zip image subset
# subset_1 if at someone wants to annotate more
os.mkdir('subset_1')
for image_name in subset_1_df.image:
    src = "/kaggle/input/happy-whale-and-dolphin/train_images/" + image_name
    dest = "/kaggle/working/subset_1/" + image_name
    shutil.copy(src, dest)
    
!zip -r /kaggle/working/subset_1.zip /kaggle/working/subset_1/ -qq
subset_1_df.to_csv("subset_1_df.csv")
```

<div class="alert alert-block alert-success"> 
That's it for now. Thanks for reading, good luck! 🍀
</div>